# Import Data (1000 obsevations only)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [2]:
true = pd.read_csv('../raw_data/True.csv', nrows = 1000)
fake = pd.read_csv('../raw_data/Fake.csv', nrows = 1000)

# Drop colums

In [3]:
true.drop(columns = ['subject','date'], inplace = True)
fake.drop(columns = ['subject','date'], inplace = True)

In [4]:
true['score'] = 1
fake['score'] = 0

In [5]:
data = pd.concat([true,fake],ignore_index=True)
data.head()

,title,text,score
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


# Removing Getty

In [6]:
# remove '/Getty Images'
stop_words =['/Getty Images']
pat = '|'.join(r"\b{}\b".format(x) for x in stop_words)
data['text'] = data['text'].str.replace(pat, '')

## Model

In [7]:
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
data['clean_text'] = data['text'].map(remove_punctuation)
data['clean_title'] = data['title'].map(remove_punctuation)

import string
def remove_punctuation(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text
data['text'] = data['text'].map(remove_punctuation)
data['title'] = data['title'].map(remove_punctuation)
def lower_case(text):
    text = text.lower()
    return text
data['text'] = data['text'].map(lower_case)
data['title'] = data['title'].map(lower_case)
def remove_numbers(text):
    text = ''.join(word for word in text if not word.isdigit())
    return text
data['text'] = data['text'].map(remove_numbers)
data['title'] = data['title'].map(remove_numbers)

In [8]:
from sklearn.model_selection import train_test_split
x = data['text']
y = data['score']

x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
# Create Pipeline
pipeline = Pipeline([
    ('cvt', CountVectorizer()),
    ('nb', MultinomialNB()),
])
# Set parameters to search
parameters = {
    'cvt__ngram_range': ((1,1), (2,2),(3,3),(4,4)),
    'nb__alpha': (0.1,1,2,5,10),}
# Perform grid search
grid_search = GridSearchCV(pipeline, parameters,
                           scoring = "accuracy",
                           refit=True, cv=5, verbose =0)
grid_search.fit(x_train,y_train)

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
grid_search.predict(x_test)

In [ ]:
grid_search.score(x_test, y_test)


In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.3)

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range= (2, 2))
x_train_bow = vectorizer.fit_transform(x_train)


In [11]:
x_test_bow = vectorizer.transform(x_test)
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha = 10)
model.fit(x_train_bow,y_train)


MultinomialNB(alpha=10, class_prior=None, fit_prior=True)

In [12]:
x_test_bow = vectorizer.transform(x_test)

In [13]:
model.score(x_test_bow,y_test)

0.9883333333333333

In [21]:
test = pd.Series(["USA in Europe"])

In [22]:
test =  vectorizer.transform(test)

In [23]:
model.predict(test)

array([0])